In [1]:
import os
import random
from PIL import Image

def get_all_files(directory):
    all_files = []
    
    # Iterate over all the directories and files within the given directory
    for root, directories, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            all_files.append(file_path)
    
    return all_files

directory = 'MiniGPT-4/RIM-ONE_DL_images/partitioned_randomly/test_set'
files = get_all_files(directory)
random.shuffle(files)

def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

loaded_images = []

for image_path in files:
    image = load_image(image_path)
    loaded_images.append(image)

In [2]:
%cd LLaVA

/home/jupyter/opthollm/LLaVA


In [4]:
import argparse
import torch

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

from PIL import Image

import requests
from PIL import Image
from io import BytesIO
from transformers import TextStreamer

False
'CUDASetup' object has no attribute 'cuda_available'


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


[2023-08-07 05:15:47,854] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


### Define Helper Methods

In [6]:

def encode_diagnosis(diagnosis):
    # could add: if contains glaucomatous and normal, then only look at first sentence
    
    if 'glaucomatous' in diagnosis.lower():
        return 1
    if 'normal' in diagnosis.lower():
        return 0
    else:
        return 2

def fetch_ground_truth(img_path):
    split_string = img_path.split("/")

    # Find the index of "glaucoma" in the split string
    try:
        split_string.index("glaucoma")
        return 1
    except:
        return 0



# get random file methods

In [9]:
model_path = "liuhaotian/LLaVA-Lightning-MPT-7B-preview"
model_base = None
image_file = "https://llava-vl.github.io/static/images/view.jpg"
num_gpus = 1
args_conv_mode = None
max_new_tokens = 512
temperature = 0.2
debug = False

### Initalize the Model

In [11]:
disable_torch_init()

model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name)

if "v1" in model_name.lower():
    conv_mode = "llava_v1"
elif "mpt" in model_name.lower():
    conv_mode = "mpt"
else:
    conv_mode = "llava_v0"

if args_conv_mode is not None and conv_mode != args_conv_mode:
    print('[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}'.format(conv_mode, args_conv_mode, args_conv_mode))
else:
    args_conv_mode = conv_mode

conv = conv_templates[args_conv_mode].copy()
if "mpt" in model_name.lower():
    roles = ('user', 'assistant')
else:
    roles = conv.roles

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
data = {'img_path': [],
        'diagnosis1' : [],
        'diagnosis2' : [],
        'diagnosis3' : [],
        'ground_truth': [],
        'llm_message1': [],
        'llm_message2': [],
        'llm_message3': []
        }

prompt1 = "Please diagnose the image as either glaucomatous or normal."
prompt2 = """You are ophthoLLM, an ophthalmologist AI assistant that provides diagnoses on fundus 
    images in order to assist doctors. You understand that it is important to recommend consulting 
    a medical professional if there is any uncertainty, and before taking any action. You give a binary, 
    one-word diagnosis on images. You either state that the image is Glaucomatous if there are signs of 
    glaucoma, or Normal if the image appears healthy. Following these instructions, and making sure to only give 
    your answer as either “Glaucomatous” or “Normal,” please diagnose the image. Make sure to include either 'Normal' or 'Glaucoma' in your answer"""
prompt3 = """You are OphthoLLM, an ophthalmology expert AI that diagnosis Glaucoma. You have been given a fundus image.

Here are some guidelines for diagnosing Glaucoma using a fundus image. 

Optic Disc Size: The size of the optic disc should be evaluated, as variations can be a normal characteristic. However, an unusually small or large optic disc may indicate specific conditions or risk factors.

Cup-to-Disc Ratio: The cup-to-disc ratio measures the size of the cup (the depression in the center of the optic nerve head) relative to the size of the entire optic disc. An increased cup-to-disc ratio may suggest glaucomatous damage.

Cup Shape: The shape of the cup should be observed, as an asymmetric or vertically elongated cup can be an indication of glaucoma.

Optic Disc Rim: The appearance of the neuroretinal rim, which surrounds the cup, is assessed. In glaucoma, this rim tends to thin and become pale or grayish.

Rim Notching: Notching or notches in the neuroretinal rim, particularly in the inferior and superior regions, can be a characteristic sign of glaucoma.

Disc Hemorrhages: Presence of hemorrhages, small bleeding spots, at or around the optic nerve head may indicate glaucomatous damage.

Nerve Fiber Layer Defects: The doctor will look for thinning or gaps in the retinal nerve fiber layer (RNFL) around the optic nerve head, which is a common early sign of glaucoma.

Vascular Changes: Changes in the blood vessels, such as vascular narrowing, crossing defects, or bayoneting, may suggest glaucoma or other optic nerve disorders.

Optic Disc Color: The color of the optic nerve head is assessed, and any abnormal discoloration, such as pallor or hyperemia, may raise suspicion of optic nerve damage.
Peripapillary Atrophy: Doctors will look for areas of atrophy (thinning) of the retinal pigment epithelium around the optic disc, which can be associated with glaucoma.

Optic Nerve Head Excavation: The depth of the optic nerve head excavation or the cup depth is evaluated, as increased cupping can indicate glaucomatous damage.

Presence of Drusen: In elderly patients, the presence of drusen, small yellowish deposits in the optic disc, should be noted, as they may mimic glaucomatous changes.
"""

prompts = [prompt1,prompt2,prompt3]




In [27]:
# try to get it to just analyze one image and then accept another
import pandas as pd

for num, image in enumerate(loaded_images):
    
    print(num)
    data['img_path'].append(files[num])
    ground_truth = fetch_ground_truth(files[num])
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()
    data['ground_truth'].append(ground_truth)
    
    for prompt_num, inp in enumerate(prompts):
        
        print(f"Image {image}, prompt#{prompt_num}")

        conv = conv_templates[args_conv_mode].copy()
        if "mpt" in model_name.lower():
            roles = ('user', 'assistant')
        else:
            roles = conv.roles

        #image = load_image(image_file)

        if image is not None:
            # first message
            if model.config.mm_use_im_start_end:
                inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
            else:
                inp = DEFAULT_IMAGE_TOKEN + '\n' + inp
            conv.append_message(conv.roles[0], inp)
        else:
            # later messages
            conv.append_message(conv.roles[0], inp)

        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()

        input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
        keywords = [stop_str]
        stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

        #print(image_tensor)
        #print(conv)
        #print(input_ids)

        #print(data)

        with torch.inference_mode():
            output_ids = model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True,
                temperature=0.2,
                max_new_tokens=1024,
                streamer=streamer,
                use_cache=True,
                stopping_criteria=[stopping_criteria])

        outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()

        conv.messages[-1][-1] = outputs
        data['llm_message'+str(prompt_num+1)].append(outputs)

        diagnosis = encode_diagnosis(outputs)

        data['diagnosis'+str(prompt_num+1)].append(diagnosis)
        
    print("Image done: " + str(files[num]))

zero_shot_data = pd.DataFrame(data)
zero_shot_data.to_csv('llava_zero_shot_big_data_trial.csv')
zero_shot_data.head()

0
Image <PIL.Image.Image image mode=RGB size=623x623 at 0x7F890B511810>, prompt#0
The image shows a close-up of a person's eye with a red and white pattern, which could be indicative of a condition called "glaucomatous." Glaucomatous is a condition where the fluid pressure inside the eye increases, causing damage to the optic nerve and leading to vision loss. However, it is essential to note that a definitive diagnosis can only be made by examining the patient in person or with a proper medical examination. The image alone does not provide enough information to confidently determine if the person has glaucomatous or not.
Image <PIL.Image.Image image mode=RGB size=623x623 at 0x7F890B511810>, prompt#1
The image shows a close-up of a person's eye, with a red spot in the center. Based on the image, it is difficult to definitively determine whether the person has Glaucomatous or not. However, the presence of a red spot in the center of the eye might be indicative of Glaucomatous, so it is i

,img_path,diagnosis1,diagnosis2,diagnosis3,ground_truth,llm_message1,llm_message2,llm_message3
0,MiniGPT-4/RIM-ONE_DL_images/partitioned_random...,1,1,2,0,The image shows a close-up of a person's eye w...,"The image shows a close-up of a person's eye, ...","In the given fundus image, the optic disc appe..."
1,MiniGPT-4/RIM-ONE_DL_images/partitioned_random...,1,2,2,0,The image shows a close-up of a person's eye w...,"The image shows a close-up of a person's eye, ...","In the given image, the fundus shows a yellowi..."
2,MiniGPT-4/RIM-ONE_DL_images/partitioned_random...,1,1,2,0,The image shows a close-up view of a person's ...,"The image is Glaucomatous, as there are signs ...","In the given fundus image, the optic disc appe..."
3,MiniGPT-4/RIM-ONE_DL_images/partitioned_random...,2,1,2,0,The image shows a close-up of a person's eye w...,"The image shows a close-up of the eye, with a ...","In the given image, the optic disc appears to ..."
4,MiniGPT-4/RIM-ONE_DL_images/partitioned_random...,1,1,2,1,The image shows a close-up view of a person's ...,"The image is Glaucomatous, as it shows signs o...","In the given image, the optic disc appears to ..."


In [25]:
import numpy as np

for key, value in data.items():
    print(key, np.shape(value))



img_path (153,)
diagnosis1 (152,)
diagnosis2 (151,)
diagnosis3 (150,)
ground_truth (152,)
llm_message1 (152,)
llm_message2 (151,)
llm_message3 (150,)


In [ ]:
while True:
        try:
            inp = input(f"{roles[0]}: ")
        except EOFError:
            inp = ""
        if not inp:
            print("exit...")
            break

        print(f"{roles[1]}: ", end="")

        if image is not None:
            # first message
            if model.config.mm_use_im_start_end:
                inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
            else:
                inp = DEFAULT_IMAGE_TOKEN + '\n' + inp
            conv.append_message(conv.roles[0], inp)
            image = None
        else:
            # later messages
            conv.append_message(conv.roles[0], inp)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()

        input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
        keywords = [stop_str]
        stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

        with torch.inference_mode():
            output_ids = model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True,
                temperature=0.2,
                max_new_tokens=1024,
                streamer=streamer,
                use_cache=True,
                stopping_criteria=[stopping_criteria])

        outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
        conv.messages[-1][-1] = outputs

        if debug:
            print("\n", {"prompt": prompt, "outputs": outputs}, "\n")


user:  Describe the image
assistant: The image features a dock on a lake, with a wooden plank extending into the water. The dock is surrounded by a beautiful, clear lake with a mountain in the background. The scene is set during the day, with the sun shining, creating a serene and picturesque atmosphere.
user:  What shouldn't I do there?
assistant: While the dock and the surrounding area are visually appealing, there are a few things you should avoid doing there. First, you should not swim in the lake, as the water might be too cold or have strong currents. Second, you should not venture too far from the dock, as the water might be too deep or have hidden obstacles that could pose a risk. Lastly, it is essential to be mindful of the surrounding environment, including the mountain and the trees, to avoid any accidents or damage to the natural landscape.
user:  thanks!
assistant: You're welcome! The image is indeed very beautiful and serene, and it's important to be aware of the surround